In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## Set-up Environment
import os
import sys

# Setup: Get notebook directory and set project root
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
os.chdir(project_root)

# Add project root to Python path for absolute imports like 'env.trading_env'
if project_root not in sys.path:
    sys.path.append(project_root)

# Print confirmation
print("📂 Notebook directory:", notebook_dir)
print("✅ Project root set to:", project_root)

# Change directory to project folder (optional)
%cd /workspaces/Strategy_PPO_Bots


📂 Notebook directory: /workspaces/Strategy_PPO_Bots
✅ Project root set to: /workspaces
/workspaces/Strategy_PPO_Bots


In [3]:
## Check to ensure in the right directory
! pwd

/workspaces/Strategy_PPO_Bots


In [4]:
## Install dependencies
! pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
## Run integration test script
#! pytest tests/integration/test_integration.py --maxfail=1 --disable-warnings -q

In [5]:
## Generate Training Data
! python data_gen/generate_data.py


Processing Ticker: NVDA
Found 1 non-intraday timeframe(s) to pre-compute: ['daily']
Found 2 intraday timeframe(s) for main processing: ['1min', '5min']

--- PASS 1: Pre-computing features (OBS_WINDOW not applied) ---
  ⚙️ Pre-computing for timeframe: daily...
    Fetching day data in 14 chunk(s)...
  ✅ Saved pre-computed features to data/NVDA_daily_precomputed.parquet

--- PASS 2: Main processing (OBS_WINDOW applied) ---
  ⚙️ Processing timeframe: 1min...
    Fetching minute data in 6 chunk(s)...
  ⚙️ Processing timeframe: 5min...
    Fetching minute data in 6 chunk(s)...

--- Final Merge ---
📉 Base timeframe determined as: 1min
  🖇️ Merging 5min features...
  🖇️ Merging pre-computed daily features from data/NVDA_daily_precomputed.parquet...
🔬 No output_columns specified. Defaulting to timestamp + generated features.
💾 Saved to data/NVDA_1min.csv
💾 Saved to data/NVDA_1min.parquet
✅ Finished processing for NVDA. Final file: NVDA_1min

Processing Ticker: @SIM
Recognized simulated ticker

In [ ]:
## ─── Plot all 1day CSVs in data/ ─────────────────────────────────────────
import glob

import matplotlib.pyplot as plt
import pandas as pd

# 1. Grab every CSV matching *_1day.csv
csv_files = glob.glob("data/*_1day.csv")
if not csv_files:
    raise FileNotFoundError("No files found: data/*_1day.csv")

# 2. Loop and plot
for file_path in sorted(csv_files):
    # Load and parse time
    df = pd.read_csv(file_path, parse_dates=[0])
    df.set_index(df.columns[0], inplace=True)

    # Extract ticker from filename
    ticker = file_path.split("/")[-1].split("_")[0]

    # Create the figure
    plt.figure(figsize=(10, 4))
    plt.plot(df.index, df["close"], linewidth=1)
    plt.title(f"{ticker} 1-Day Close Price")
    plt.xlabel("Time")
    plt.ylabel("Close Price")
    plt.tight_layout()
    plt.show()

In [ ]:
## Run Integration Test for PPO
# %run train/train_ppo.py --test


In [ ]:
# import torch

# print("CUDA Available:", torch.cuda.is_available())
# print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


In [ ]:
# Run PPO training using files in `data/` directory
%run train/train_ppo.py

In [ ]:
#! ls ../debug_rewards.log
